In [1]:
#setup, importing modules
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Machine learning 
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn import metrics

In [2]:
#data
from tensorflow.keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

2024-03-15 17:10:03.902597: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-15 17:10:03.907335: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-15 17:10:03.958617: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-15 17:10:04.991545: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
#functions
#greyscale



def grey_normalize_image(image,bins = 255):

    """ function for greyscaling in an image file, taking it's color histogram and normailizing it. """
    #greyscale
    greyed_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #normalize
    hist = cv2.calcHist([greyed_image],[0],None,[bins],[0,256])
    normalized_hist = cv2.normalize(hist, hist, 0, 1.0, cv2.NORM_MINMAX)
    #squeeze a dimension out

    normalized_hist = np.squeeze(normalized_hist)


    return(normalized_hist)

def show_col_channels(img, bins = 255):
    color = ('b','g','r')
    for i,col in enumerate(color):
        histr = cv2.calcHist([img],[i],None,[bins],[0,256])
        plt.plot(histr,color = col)
        plt.xlim([0,bins])
    plt.show()

def ch3_normalize_image(image,bins = 255):

    """ function for an image file, taking it's color histograms and normailizing them. """
    color_histogram_list = []
    for channel in range(0,3):

        #normalize
        hist = cv2.calcHist([image],[channel],None,[bins],[0,256])
        normalized_hist = cv2.normalize(hist, hist, 0, 1.0, cv2.NORM_MINMAX)
        #squeeze a dimension out
        normalized_hist = np.squeeze(normalized_hist)
        color_histogram_list.append(normalized_hist)
    
    #make it one array
    color_histogram = np.concatenate(np.array(color_histogram_list))


    return(color_histogram)


import matplotlib as mpl
# jimshow commisioned from utils
def jimshow(image, title=False):
    """imshow with matplotlib dependencies 
    """
    # Acquire default dots per inch value of matplotlib
    dpi = mpl.rcParams['figure.dpi']

    height, width, depth = image.shape
    figsize = width / float(dpi), height / float(dpi)
    
    plt.figure(figsize=figsize)
    
    if depth == 1:
        plt.imshow(image, cmap='gray')
    else:
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
      
    if title:
        plt.title(title)
    plt.axis('off')
    
    plt.show()

def jimshow_channel(image, title=False):
    """
    Modified jimshow() to plot individual channels
    """
    # Acquire default dots per inch value of matplotlib
    dpi = mpl.rcParams['figure.dpi']

    height, width = image.shape
    figsize = width / float(dpi), height / float(dpi)
    
    plt.figure(figsize=figsize)
    
    plt.imshow(image, cmap='gray')
      
    if title:
        plt.title(title)
    plt.axis('off')
    
    plt.show()

In [33]:

def grey_normalize_image(image):

    """ function for greyscaling in an image file, and normailizing it. """
    #greyscale
    greyed_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #normalize
    normalized = greyed_image/255.0
    #reshape
    normalized = normalized.reshape(-1, 1024)
    #squeeeze out a dimension
    normalized = np.squeeze(normalized)


    return(normalized)

In [34]:
X_train_normalized_grey =[]
for image in X_train:
    temp_norm_im = grey_normalize_image(image)
    X_train_normalized_grey.append(temp_norm_im)

X_test_normalized_grey =[]
for image in X_test:
    temp_norm_im = grey_normalize_image(image)
    X_test_normalized_grey.append(temp_norm_im)
    


In [37]:
classifier = LogisticRegression(tol=0.2, 
                         solver='saga',
                         multi_class='multinomial',
                         random_state = 42).fit(X_train_normalized_grey, y_train)
#predict
prediction = classifier.predict(X_test_normalized_grey)
#conf_matrix
cm = np.array2string(metrics.confusion_matrix(y_test,prediction))
#report
cr = metrics.classification_report(y_test, prediction)

/home/ucloud/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
classifier = LogisticRegression(random_state=42).fit(X_train_ch3, y_train)
#predict
prediction = classifier.predict(X_test_ch3)
#conf_matrix
cm = np.array2string(metrics.confusion_matrix(y_test,prediction))
#report
cr = metrics.classification_report(y_test, prediction)

/home/ucloud/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ucloud/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
f = open('../out/log_long_tol_report.txt', 'w')
f.write('Logistic Classifier output\n\nClassification Report\n\n{}\n\nConfusion Matrix\n\n{}\n'.format(cr, cm))
f.close()